# Transform GeoJSON Properties into Records

In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from collections import Counter

import plotly.graph_objs as go
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, json
import urllib.request 

brd_count = {
    'bin': 0,    'bowl': 0, 
    'bucket': 0, 'cup': 0,
    'tire': 0,   'pottedplant': 0, 
    'jar': 0,    'vase': 0
}

## Sameple data in GeoJSON file

In [2]:
with open(os.path.join('..','data','breeding-sites','geojson','Krabi-detection.geojson'), 'r') as file:
    data = json.load(file)

print(json.dumps(data['features'][0], ensure_ascii=False, indent=4))

{
    "geometry": {
        "coordinates": [
            99.02596777,
            7.89442744
        ],
        "type": "Point"
    },
    "type": "Feature",
    "properties": {
        "province": "กระบี่",
        "brd_sites": {
            "0": {
                "count": {
                    "bucket": 1
                },
                "sum_each": 1
            },
            "216": {
                "count": {
                    "bucket": 1
                },
                "sum_each": 1
            }
        },
        "district": "เหนือคลอง",
        "sum": 2,
        "subdist": "เหนือคลอง",
        "date": {
            "month": "03",
            "year": "2016"
        },
        "image_name": "7.89442744_99.02596777"
    }
}


In [15]:
rows = []
for i, feature in enumerate(data['features']):
    
    row = feature['properties'].copy()
    
    lng, lat = feature['geometry']['coordinates']
    row['lng'], row['lat'] = lng, lat
    
    row['date'] = row['date']['year']+'-'+row['date']['month']
    
    for degree in row['brd_sites']:
        
        detected_brd = row['brd_sites'][degree]['count']

        for cls in brd_count:
            if cls not in row:
                row[cls] = 0
            
            if cls in detected_brd:
                row[cls] += detected_brd[cls]
                
    _= row.pop('brd_sites')
#     _= row.pop('directory')
    _= row.pop('image_name')

    rows.append(row)
    
df = pd.DataFrame.from_dict(rows)
df = df.drop('province', axis=1)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m')
df = df.set_index('date')
df = df.sort_index()

print('Most common year:', Counter(list(df.index.year)).most_common())

df = df['2016']
df.head()
df.tail()
print('Total:',len(df))

Most common year: [(2016, 16019), (2017, 560), (1970, 4), (2015, 4), (2018, 1), (2014, 1)]


,bin,bowl,bucket,cup,district,jar,lat,lng,pottedplant,subdist,sum,tire,vase
date,,,,,,,,,,,,,
2016-01-01,0,0,2,0,เมืองกระบี่,0,8.006397,98.840986,1,อ่าวนาง,3,0,0
2016-01-01,0,0,0,0,เมืองกระบี่,0,8.019386,98.837395,1,อ่าวนาง,1,0,0
2016-01-01,0,0,0,0,เมืองกระบี่,0,8.012092,98.839691,2,อ่าวนาง,2,0,0
2016-01-01,0,0,0,0,เมืองกระบี่,0,8.011447,98.838362,1,อ่าวนาง,1,0,0
2016-01-01,0,0,1,0,เมืองกระบี่,0,8.010250,98.841269,0,อ่าวนาง,1,0,0


,bin,bowl,bucket,cup,district,jar,lat,lng,pottedplant,subdist,sum,tire,vase
date,,,,,,,,,,,,,
2016-12-01,0,0,1,0,ลำทับ,0,8.189570,99.323854,0,ลำทับ,1,0,0
2016-12-01,0,0,1,0,ลำทับ,0,8.186216,99.321156,0,ลำทับ,1,0,0
2016-12-01,0,0,1,0,ลำทับ,0,8.177941,99.314485,0,ลำทับ,1,0,0
2016-12-01,0,0,0,0,ลำทับ,0,8.192055,99.325797,1,ลำทับ,1,0,0
2016-12-01,0,0,1,0,ลำทับ,0,8.168844,99.309304,0,ลำทับ,1,0,0


Total: 16019


[(2016, 16019), (2017, 560), (1970, 4), (2015, 4), (2018, 1), (2014, 1)]

## Load subdistrict polygons

In [7]:
with open(os.path.join('..','data','shapefiles','Krabi-subdisticts.geojson'), 'r') as file:
    data_polygon = json.load(file)

In [8]:
data_polygon['features'][0]['properties']

{'AP_EN': 'Ko Lanta',
 'AP_IDN': '8103',
 'AP_TN': 'เกาะลันตา',
 'PV_EN': 'Krabi',
 'PV_IDN': '81',
 'PV_TN': 'กระบี่',
 'TB_EN': 'Ko Lanta Yai',
 'TB_IDN': '810301',
 'TB_TN': 'เกาะลันตาใหญ่'}

## Sum number of breeding sites in each subdistrict

In [10]:
df_sum = []
polygons = []
province = 'กระบี่'
for i, feature in enumerate(data_polygon['features']):
    prop = feature['properties']
    
    addrcode = prop['TB_IDN']
    province, district, subdist = prop['PV_TN'], prop['AP_TN'], prop['TB_TN']
    province_en, district_en, subdist_en = prop['PV_EN'], prop['AP_EN'], prop['TB_EN']
    value = df[(df.district == district) & (df.subdist == subdist)].copy()
    
    if len(value) == 0:
        continue
    
    total = 0
    tmp_sum = []
    for cls in brd_count:
        tmp_sum.append(value[cls].sum())
        total += value[cls].sum()

    df_sum.append([Counter(list(value.index)).most_common(1)[0][0], addrcode,
                   province, district, subdist,
                   province_en, district_en, subdist_en,
                  ]+tmp_sum+[total])
    
df_sum = pd.DataFrame.from_records(df_sum)
df_sum.columns = ['date','addrcode','province','district','subdist',
                  'province_en','district_en','subdist_en',
                  'bin','bowl', 'bucket','cup','jar',
                  'pottedplant','tire','vase','total']
df_sum = df_sum.set_index('date')
df_sum.drop_duplicates(inplace=True)

df_sum.head()
df_sum.tail()
print('Total:',len(df_sum))

df_sum.to_csv(os.path.join('..','data','breeding-sites','csv','Krabi-detection.csv'))

,addrcode,province,district,subdist,province_en,district_en,subdist_en,bin,bowl,bucket,cup,jar,pottedplant,tire,vase,total
date,,,,,,,,,,,,,,,,
2016-02-01,810503,กระบี่,อ่าวลึก,นาเหนือ,Krabi,Ao Luek,Na Nuea,307,9,104,1,32,1,38,10,502
2016-02-01,810507,กระบี่,อ่าวลึก,เขาใหญ่,Krabi,Ao Luek,Khao Yai,253,16,146,2,80,0,55,3,555
2016-03-01,810604,กระบี่,ปลายพระยา,คีรีวง,Krabi,Plai Phraya,Khiri Wong,160,35,175,1,87,2,31,5,496
2016-02-01,810205,กระบี่,เขาพนม,หน้าเขา,Krabi,Khao Phanom,Na Khao,263,21,112,1,33,0,28,2,460
2016-02-01,810508,กระบี่,อ่าวลึก,คลองยา,Krabi,Ao Luek,Khlong Ya,214,18,143,1,48,0,36,2,462


,addrcode,province,district,subdist,province_en,district_en,subdist_en,bin,bowl,bucket,cup,jar,pottedplant,tire,vase,total
date,,,,,,,,,,,,,,,,
2016-03-01,810404,กระบี่,คลองท่อม,ทรายขาว,Krabi,Khlong Thom,Sai Khao,643,9,268,3,92,2,77,17,1111
2016-04-01,810305,กระบี่,เกาะลันตา,ศาลาด่าน,Krabi,Ko Lanta,Sala Dan,251,7,172,7,15,0,31,10,493
2016-02-01,810603,กระบี่,ปลายพระยา,เขาต่อ,Krabi,Plai Phraya,Khao To,347,12,151,2,10,1,50,6,579
2016-03-01,810601,กระบี่,ปลายพระยา,ปลายพระยา,Krabi,Plai Phraya,Plai Phraya,605,44,378,5,102,4,73,8,1219
2016-02-01,810602,กระบี่,ปลายพระยา,เขาเขน,Krabi,Plai Phraya,Khao Khen,270,35,139,2,38,0,24,1,509


Total: 47
